In [ ]:
import re
import json
import pandas as pd
from datetime import datetime



In [ ]:
file_path = "2025-11-11_Umsteigehinweise_EVB_Beispiel.txt"

data = []
pattern = re.compile(r"<K(.*?)>")

with open(file_path, encoding="utf-8") as f:
    for line in f:
        match = pattern.search(line)
        if not match:
            continue

        payload = match.group(1)
        fields = payload.split("#")

        for i in range(len(fields) - 15):
            if re.match(r"^\d{5,6}$", fields[i]):  # Start einer Fahrt erwartet werden 5 oder 6 stellige IDs
                try:
                    stop_id = fields[i]
                    stop_name = fields[i + 1]
                    mode = fields[i + 2]  # BUS, Tram, etc.
                    line_no = fields[i + 6] if len(fields) > i + 6 else None

                    # Ziel im JSON-Feld
                    json_field = fields[i + 11] if len(fields) > i + 11 else ""
                    final_json = None
                    if "{" in json_field and "}" in json_field:
                        try:
                            final_json = json.loads(json_field)
                        except json.JSONDecodeError:
                            pass

                    dep_time = fields[i + 12] if len(fields) > i + 12 else None
                    arr_time = fields[i + 13] if len(fields) > i + 13 else None

                    data.append({
                        "StopID": stop_id,
                        "Haltestelle": stop_name,
                        "Verkehrsmittel": mode,
                        "Linie": line_no,
                        "Abfahrt": dep_time,
                        "Ankunft": arr_time,
                        "Ziel": final_json["Final"]["Text"] if final_json else None,
                    })
                except Exception:
                    continue

# In DataFrame konvertieren
df = pd.DataFrame(data)



In [4]:
df

,StopID,Haltestelle,Verkehrsmittel,Linie,Abfahrt,Ankunft,Ziel
0,114562,"Bremen, Am Lehester Deich",BUS,1670,2025-11-11-08:04:00,2025-11-11-08:11:45,Bremen Hbf
1,114561,"Bremen, Am Lehester Deich",,,1630,630,None
2,114562,"Bremen, Am Lehester Deich",,,1670,670,None
3,114561,"Bremen, Am Lehester Deich",,,1670,670,None
4,114562,"Bremen, Am Lehester Deich",,,1670,670,None
5,114561,"Bremen, Am Lehester Deich",,,1670,670,None
6,114561,"Bremen, Am Lehester Deich",,,1630,630,None
7,114552,"Bremen, Kopernikusstraße",BUS,1670,2025-11-11-08:08:00,2025-11-11-08:15:45,Bremen Hbf
8,114551,"Bremen, Kopernikusstraße",,,1630,630,None
9,114552,"Bremen, Kopernikusstraße",,,1670,670,None


In [ ]:
# Nur sinnvolle Einträge behalten
df = df.dropna(subset=["Haltestelle", "Abfahrt", "Ziel"])
df = df.drop_duplicates()

# Abfahrtszeit als Datetime-Objekt
df["Abfahrt_dt"] = pd.to_datetime(df["Abfahrt"], errors="coerce")
df = df.sort_values(by=["Haltestelle", "Abfahrt_dt"]).reset_index(drop=True)

# Umstiegserkennung + Umstiegszeit
df["Umstieg_möglich"] = False
df["Umstiegszeit_min"] = None

for stop in df["Haltestelle"].unique():
    sub = df[df["Haltestelle"] == stop].sort_values("Abfahrt_dt")
    for i in range(1, len(sub)):
        delta = (sub.iloc[i]["Abfahrt_dt"] - sub.iloc[i - 1]["Abfahrt_dt"]).total_seconds() / 60
        if 0 < delta <= 10:
            df.loc[sub.index[i], "Umstieg_möglich"] = True
            df.loc[sub.index[i - 1], "Umstieg_möglich"] = True
            df.loc[sub.index[i], "Umstiegszeit_min"] = round(delta, 1)
            df.loc[sub.index[i - 1], "Umstiegszeit_min"] = round(delta, 1)

# Aufräumen
df = df.drop(columns=["Abfahrt_dt"])

# Ergebnis speichern
df.to_csv("evb_parsed_umstieg_zeiten.csv", index=False, encoding="utf-8-sig")
print(df.head(15))
print("\n✅ Ergebnis gespeichert als 'evb_parsed_umstieg_zeiten.csv'")

In [2]:
df.head()

,StopID,Haltestelle,Verkehrsmittel,Linie,Abfahrt,Ankunft,Ziel,Umstieg_möglich,Umstiegszeit_min
0,114562,"Bremen, Am Lehester Deich",BUS,1670,2025-11-11-08:04:00,2025-11-11-08:11:45,Bremen Hbf,False,None
1,159902,"Bremen, Hochschulring",BUS,1670,2025-11-11-08:10:00,2025-11-11-08:17:45,Bremen Hbf,False,None
2,114552,"Bremen, Kopernikusstraße",BUS,1670,2025-11-11-08:08:00,2025-11-11-08:15:45,Bremen Hbf,False,None
